In [1]:
import glob
from collections import Counter
import string
import os
from tqdm import tqdm
import pickle

from fuzzywuzzy import process

from config import Config
from risk_detection.utils import get_file_name_without_ext, window
from risk_detection.preprocessing.report_parser import report_info_from_risk_path
from risk_detection.analysis.keyword_extraction import Keywords

In [2]:
def find_cluster_matches(word_prev_cluster, prev_clusters, word_cluster, curr_clusters):
    matches = dict()

    for cluster_num, cluster_words in curr_clusters.items():
        prev_cluster_nums = list()
        for word in cluster_words:
            words = process.extract(word, word_prev_cluster.keys(), limit=5)
            prev_cluster_nums.extend([word_prev_cluster[w[0]] for w in words])

        counts = Counter(prev_cluster_nums)
        # Check if we have a perfect match
        for prev_cluster_num, _ in counts.most_common():
            if set(cluster_words) == set(prev_clusters[prev_cluster_num]):
                matches[cluster_num] = prev_cluster_num
                continue
        
        prev_closest_cluster_num = counts.most_common()[0][0]
        common_words = set(cluster_words).intersection(set(prev_clusters[prev_closest_cluster_num]))
        if len(common_words) >= (min(len(cluster_words), len(prev_clusters[prev_closest_cluster_num])) // 2):
            matches[cluster_num] = prev_closest_cluster_num
    
    return matches

In [16]:
def find_cluster_matches_semantic(word_prev_cluster, prev_clusters, word_cluster, curr_clusters):
    matches = dict()
    
    prev_corpus = list(word_prev_cluster.keys())
    curr_corpus = list(word_cluster.keys())
    curr_corpus_lookup = {k: v for v, k in enumerate(curr_corpus)}
    
    prev_emb = embedder.encode(prev_corpus)
    curr_emb = embedder.encode(curr_corpus)
    semantic_matches = util.semantic_search(curr_emb, prev_emb, top_k=5)

    for cluster_num, cluster_words in curr_clusters.items():
        prev_cluster_nums = list()
        for word in cluster_words:
            matches_word_indices = semantic_matches[curr_corpus_lookup[word]]
            prev_cluster_nums.extend([word_prev_cluster[prev_corpus[w['corpus_id']]] for w in matches_word_indices])

        counts = Counter(prev_cluster_nums)
        # Check if we have a perfect match
        for prev_cluster_num, _ in counts.most_common():
            if set(cluster_words) == set(prev_clusters[prev_cluster_num]):
                matches[cluster_num] = prev_cluster_num
                continue
        
        prev_closest_cluster_num = counts.most_common()[0][0]
        common_words = set(cluster_words).intersection(set(prev_clusters[prev_closest_cluster_num]))
        if len(common_words) >= (min(len(cluster_words), len(prev_clusters[prev_closest_cluster_num])) // 2):
            matches[cluster_num] = prev_closest_cluster_num
    
    return matches

In [3]:
cluster_files = glob.glob(os.path.join(Config.keywords_dir(), 'yearly_clusters', '*.pickle'))

In [4]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('stsb-roberta-base')

In [18]:
for prev_file_n, curr_file_n in window(sorted(cluster_files, key=lambda x: int(get_file_name_without_ext(x)))):
    with open(prev_file_n, 'rb') as prev_file:
        word_prev_cluster, prev_clusters = pickle.load(prev_file)
    with open(curr_file_n, 'rb') as curr_file:
        word_cluster, curr_clusters = pickle.load(curr_file)
    
    matches = find_cluster_matches_semantic(word_prev_cluster, prev_clusters, word_cluster, curr_clusters)
    import pdb; pdb.set_trace()

> <ipython-input-18-6700eb184d03>(1)<module>()
----> 1 for prev_file_n, curr_file_n in window(sorted(cluster_files, key=lambda x: int(get_file_name_without_ext(x)))):
      2     with open(prev_file_n, 'rb') as prev_file:
      3         word_prev_cluster, prev_clusters = pickle.load(prev_file)
      4     with open(curr_file_n, 'rb') as curr_file:
      5         word_cluster, curr_clusters = pickle.load(curr_file)



ipdb>  n


> <ipython-input-18-6700eb184d03>(2)<module>()
      1 for prev_file_n, curr_file_n in window(sorted(cluster_files, key=lambda x: int(get_file_name_without_ext(x)))):
----> 2     with open(prev_file_n, 'rb') as prev_file:
      3         word_prev_cluster, prev_clusters = pickle.load(prev_file)
      4     with open(curr_file_n, 'rb') as curr_file:
      5         word_cluster, curr_clusters = pickle.load(curr_file)



ipdb>  prev_file_n


'c:\\machine_learning\\10k-emerging-risk-detection\\models\\keywords\\yearly_clusters\\2004.pickle'


...

6518


ipdb>  qqq


*** NameError: name 'qqq' is not defined


ipdb>  q


BdbQuit: 

In [12]:
for prev_file_n, curr_file_n in window(sorted(cluster_files, key=lambda x: int(get_file_name_without_ext(x)))):
    with open(prev_file_n, 'rb') as prev_file:
        word_prev_cluster, prev_clusters = pickle.load(prev_file)
    with open(curr_file_n, 'rb') as curr_file:
        word_cluster, curr_clusters = pickle.load(curr_file)
    
    matches = find_cluster_matches(word_prev_cluster, prev_clusters, word_cluster, curr_clusters)

> <ipython-input-12-ac2f40f74fd1>(1)<module>()
----> 1 for prev_file_n, curr_file_n in window(sorted(cluster_files, key=lambda x: int(get_file_name_without_ext(x)))):
      2     with open(prev_file_n, 'rb') as prev_file:
      3         word_prev_cluster, prev_clusters = pickle.load(prev_file)
      4     with open(curr_file_n, 'rb') as curr_file:
      5         word_cluster, curr_clusters = pickle.load(curr_file)



ipdb>  matches


{29: 7, 49: 13, 25: 15}


ipdb>  curr_cluster[29]


*** NameError: name 'curr_cluster' is not defined


...

['us housing finance sector', 'us housing market', 'us housing prices']


ipdb>  c



Program interrupted. (Use 'cont' to resume).
> c:\machine_learning\10k-emerging-risk-detection\venv\lib\site-packages\fuzzywuzzy\fuzz.py(139)_token_set()
    137     tokens2 = set(p2.split())
    138 
--> 139     intersection = tokens1.intersection(tokens2)
    140     diff1to2 = tokens1.difference(tokens2)
    141     diff2to1 = tokens2.difference(tokens1)

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


KeyboardInterrupt: 

ipdb>  q
